In [1]:
pacman::p_load(data.table,ggplot2,ggalt,ggthemes,lubridate,dplyr,RColorBrewer,patchwork,ggupset)

In [2]:
wrld <- map_data("world")

gg <- ggplot()
gg <- gg + geom_map(data=wrld, map=wrld,
                    aes(x=long, y=lat, map_id=region),
                    color="#2b2b2b", fill="#2b2b2b",size=0.15, fill=NA)
gg <- gg + coord_proj("+proj=robin +lon_0=0 +x_0=0 +y_0=0 +ellps=WGS84 +datum=WGS84 +units=m +no_defs")
gg <- gg + theme_map()

Warning message:
“Duplicated aesthetics after name
standardisation: fill”
Warning message:
“Using `size` aesthetic for lines
was deprecated in ggplot2 3.4.0.
ℹ Please use `linewidth` instead.”
Warning message in geom_map(data = wrld, map = wrld, aes(x = long, y = lat, map_id = region), :
“Ignoring unknown aesthetics: x
and y”


In [3]:
norberg_intercept<-function(a,b,width,opt,intercept_1=TRUE){
    if (intercept_1) {
        return(width/2+opt)
    } else {
        return(-width/2+opt)
    }
}

In [4]:
list_of_temp_opt=c(0.0,3.5,7,10.5,14,17.5,21,24.5,28,31.5)
list_of_a=rev(sort(rep(c(0.30,0.29,0.28,0.27,0.26,0.25),10)))
list_of_b=rep(0.066171077,60)
list_of_w=sort(rep(c(16,18,20,24,28,32),10))
tracer=c(1:60)
tracer_fr_more_widths=data.frame("Tracer"=tracer,
                                 "w"=list_of_w,
                                 "a"=list_of_a,
                                 "b"=list_of_b,
                                 "opt"=list_of_temp_opt,
                                 "Scenario"="Generalist-Specialist")%>%
    rowwise() %>%
    dplyr::mutate(LowIntercept=min(norberg_intercept(a,b,w,opt,TRUE),
                                   norberg_intercept(a,b,w,opt,FALSE)),
                  HighIntercept=max(norberg_intercept(a,b,w,opt,TRUE),
                                   norberg_intercept(a,b,w,opt,FALSE)))

In [5]:
ALL_EHUX = read.csv('../data/00-ehux-all-strains.csv')
## read in Harriet's colors

strain_color_frame_harriet = data.frame("Strains_full"=c("RCC6856","RCC6071","RCC1212","RCC3963",
                                                   "RCC874","CCMP1280","RCC4567","RCC914","RCC3492",
                                                   "CCMP371","RCC1215","CCMP377","RCC1239",
                                                   "RCC1222","RCC1256","CCMP374","CCMP379","CCMP1516",
                                                   "CCMP2090","CCMP375"),
                                        "Strains"=c("6856","6071","1212","3963",
                                                   "874","1280","4567","914","3492",
                                                   "371","1215","377","1239",
                                                   "1222","1256","374","379","1516",
                                                   "2090","375"),
                                        "Colors"=c("#4443c6","#c688ff","#7d328b","#da1b9d",
                                                  "#f87fa2","#b90033","#ff5755","#c31d0b",
                                                  "#ff6235","#ffc10c","#e4e366","#5a9f00",
                                                  "#60b574","#00b9bf","#0194e3","#C2C95E",
                                                  "#5EC9C9","#000000","#DBDBDB","#013220"))

simple_frame = ALL_EHUX %>%
    dplyr::select("Latitude","Longitude","X") %>%
    dplyr::bind_rows(data.frame("X"=c("CCMP371","CCMP1280","CCMP375","CCMP377",
                       "CCMP1516","CCMP2090","CCMP379","RCC4567","CCMP371","CCMP374"),
                "Latitude"=c(32,-12,32,43,-2.67,-2.67,50.1669,-11.48,32,42.5),
                "Longitude"=c(-62,-35,-62,-68,-82.72,-82.72,-4.2504,-25.05,-62,-69)))

color_frame=simple_frame %>%
    dplyr::left_join(strain_color_frame_harriet,by=c("X"="Strains_full"))

In [ ]:
morewidthsdir="../data/Darwin_Simulations/ModWidths_FixedHet_MostConservative"

temps_ModWidths_ConRange_MoreWidths=data.frame(fread(file.path(morewidthsdir,
                                    "tave.ScenarioB_no_low_opt.revise.concatenated_temp_phys_tave_monthly_all_biomass.csv")))

temps_ModWidths_ConRange_MoreWidths=temps_ModWidths_ConRange_MoreWidths %>%
    dplyr::mutate(DateTime = strptime(Time, format='%Y-%b-%d %H:%M:%S'))

all_all_depths_ModWidths_ConRange_MoreWidths=data.frame(fread(file.path(morewidthsdir,
                                                         "ScenarioB_depths_2Oct2023_temp_tave_monthly_all_biomass.csv")))

all_all_depths_ModWidths_ConRange_MoreWidths = all_all_depths_ModWidths_ConRange_MoreWidths %>%
    dplyr::mutate(DateTime = strptime(Time, format='%Y-%b-%d %H:%M:%S'))

merged_all_w_temp_scen=all_all_depths_ModWidths_ConRange_MoreWidths %>%
    dplyr::left_join(tracer_fr_more_widths, by=c("Tracer"))

topt_dict=merged_all_w_temp_scen

duplicates_dropped=topt_dict %>% dplyr::distinct(Latitude,Longitude,DateTime,w,opt,
                                                 biomass)

# filter based on time
duplicates_dropped_filt = duplicates_dropped %>% 
    dplyr::filter(DateTime>as.POSIXct("2008-12-01",format="%Y-%m-%d"))

# get total biomass of all time in last year of sim
total_biomass=duplicates_dropped_filt %>%
    dplyr::group_by(Latitude,Longitude) %>%
    dplyr::summarize(total_biomass=sum(biomass,na.rm=TRUE))

# summarize weighted biomass in final year of simulation
duplicates_dropped_filt_summary = duplicates_dropped_filt %>% 
    dplyr::left_join(total_biomass,by=c("Latitude","Longitude")) %>%
    dplyr::mutate(WeightedWidths=(w*biomass)/total_biomass) %>%
    dplyr::group_by(Latitude,Longitude) %>%
    dplyr::summarize(WeightedWidth=sum(WeightedWidths,na.rm=TRUE))

total_biomass=duplicates_dropped_filt %>%
    dplyr::group_by(Latitude,Longitude,DateTime) %>%
    dplyr::summarize(total_biomass=sum(biomass,na.rm=TRUE))

## filter timepoints and tracers w/ biomass > 0.01
get_topts_higher_biomass = duplicates_dropped_filt%>% 
    dplyr::left_join(total_biomass,by=c("Latitude","Longitude","DateTime"))%>%
    dplyr::mutate(Longitude=(Longitude + 180) %% 360 - 180) %>%
    dplyr::group_by(Latitude,Longitude,DateTime) %>% 
    dplyr::filter(biomass/total_biomass > 0.01) %>%
    dplyr::ungroup() %>% 
    dplyr::distinct(Latitude,Longitude,w,opt) %>%
    dplyr::left_join(tracer_fr_more_widths)

In [ ]:
## read in experimental Norberg params
## get x intercepts of experimental Norberg
experiment_widths=read.csv("darwin-processing-scripts/num_coexisting_widths/all_params_experiments_15Dec_fromfinal-code-notebooks.csv") %>%
    dplyr::rowwise() %>%
    dplyr::mutate(LowIntercept=min(norberg_intercept(a,b,w,opt,TRUE),
                                   norberg_intercept(a,b,w,opt,FALSE)),
                  HighIntercept=max(norberg_intercept(a,b,w,opt,TRUE),
                                   norberg_intercept(a,b,w,opt,FALSE))) %>%
    dplyr::mutate(LowIntercept=case_when(Strain=="374" ~ 4,
                                         #Strain=="6071" ~ 1,
                                         TRUE ~ LowIntercept)) %>%
    tidyr::pivot_longer(cols=c("LowIntercept","HighIntercept"),
                        names_to="LowOrHigh",values_to="ExpIntercept")

get_topts_higher_biomass["key"] = "noval"


## merge modeled Topt values with experimental ones
all_means=get_topts_higher_biomass %>%
    tidyr::pivot_longer(cols=c("LowIntercept","HighIntercept"),
                        names_to="LowOrHigh",values_to="Intercept") %>%
    dplyr::group_by(Latitude,Longitude,LowOrHigh) %>% # Remove LowOrHigh to get a distribution of both values
    dplyr::summarize(MeanIntercept=mean(Intercept,na.rm=TRUE),
                     StdIntercept=sd(Intercept,na.rm=TRUE))%>%
    dplyr::mutate(StdIntercept=case_when(is.na(StdIntercept)~1,
                                        TRUE ~StdIntercept))

all_means["key"] = "noval"
merged_df_all = all_means %>% 
    dplyr::left_join(experiment_widths%>%dplyr::select(ExpIntercept,Strain,LowOrHigh))

In [ ]:
## simple pdf protocol. the two populations are independent due to flexible width.
merged_df_all = merged_df_all %>%rowwise() %>%
    dplyr::mutate(Probability=dnorm(ExpIntercept,mean=MeanIntercept,
                                    sd=StdIntercept)/
                              dnorm(MeanIntercept,mean=MeanIntercept,
                                    sd=StdIntercept))
                  
merged_df_all_plot=merged_df_all %>% tidyr::pivot_wider(id_cols = c("Latitude","Longitude","Strain"),
                                     names_from="LowOrHigh",values_from="Probability") %>%
    dplyr::rowwise()%>%
    dplyr::mutate(Probability=HighIntercept*LowIntercept)

In [ ]:
counted_strains=merged_df_all_plot %>% dplyr::filter(Probability > 0.05) %>%
    dplyr::distinct(Strain,Latitude,Longitude) %>%
    dplyr::group_by(Latitude,Longitude) %>% 
    dplyr::summarize(NumStrains = length(unique(Strain)),
                     ListStrains = paste0(sort(Strain),collapse="_")) %>%
    dplyr::arrange(desc(NumStrains))

In [ ]:
map_overlap_teal = gg + geom_point(data=counted_strains %>% 
    dplyr::mutate(Longitude=(Longitude + 180) %% 360 - 180),
    aes(x=Longitude, y = Latitude, color = factor(NumStrains))) + 
    theme(legend.position = "bottom")+ geom_map(data=wrld, map=wrld,
                    aes(x=long, y=lat, map_id=region),
                    color="#2b2b2b", fill="#2b2b2b",size=0.15, fill=NA) + 
    scale_color_manual(values=rev(hcl.colors(palette = "ag_Sunset",n=12)),name="Number of strains with\n>=5% probability")+ 
    guides(color = guide_legend(override.aes = list(size = 5)))+
    geom_point(data=color_frame %>% dplyr::filter(Strains%in%merged_df_all$Strain) %>%
                   dplyr::distinct(Strains,Latitude,Longitude,Colors),
               aes(y=Latitude,x=Longitude),pch=23,
               fill=(color_frame %>% dplyr::filter(Strains%in%merged_df_all$Strain)%>%
                     dplyr::distinct(Strains,Latitude,Longitude,Colors))$Colors,
               color="black",size=5) +
    theme(strip.text = element_text(
    size = 16),strip.background = element_rect( colour="black", fill="white" ))
map_overlap_teal

In [ ]:
upset_plot_strains = merged_df_all_plot %>% dplyr::filter(Probability > 0.05) %>%
    dplyr::distinct(Strain,Latitude,Longitude) %>%
    dplyr::group_by(Latitude,Longitude) %>% 
    dplyr::summarize(NumStrains = length(unique(Strain)),
                     ListStrains = list(unique(as.character(Strain)))) %>%
    dplyr::arrange(desc(NumStrains))%>%
  ggplot(aes(x=ListStrains)) +
    geom_bar(fill="black") +
    scale_x_upset(n_intersections = 40) + ylab("Number of grid points\nwithin 5% probability") + 
    xlab("") + theme_bw(base_size=12)
upset_plot_strainss

In [ ]:
strain="6071"
strain_6071=gg + geom_point(data=merged_df_all_plot %>% 
    dplyr::filter(Strain==as.numeric(strain)) %>%
    dplyr::mutate(Strains=as.character(Strain))%>%
    dplyr::mutate(Longitude=(Longitude + 180) %% 360 - 180) %>%
    dplyr::left_join(strain_color_frame_harriet),
    aes(x=Longitude, y = Latitude, color = Probability)) + 
    scale_color_gradient(low="white",high=(strain_color_frame_harriet %>% 
                                           dplyr::filter(Strains==strain))$Colors[1],
                         na.value="black",name="probability")+
    theme(legend.position = "bottom")+ geom_map(data=wrld, map=wrld,
                    aes(x=long, y=lat, map_id=region),
                    color="#2b2b2b", fill="#2b2b2b",size=0.15, fill=NA) +        
    facet_wrap(~Strain)+
    geom_point(data=color_frame%>%dplyr::filter(Strains==strain),
               aes(y=Latitude,x=Longitude),pch=23,
               fill=(color_frame%>%dplyr::filter(Strains==strain))$Colors[1],
               color="black",size=5) +
  theme(strip.text = element_text(
    size = 16),strip.background = element_rect( colour="black", fill="white" ))
strain_6071

In [ ]:
wrld <- map_data("world")

gg <- ggplot()
gg <- gg + geom_map(data=wrld, map=wrld,
                    aes(x=long, y=lat, map_id=region),
                    color="#2b2b2b", fill="#2b2b2b",size=0.15, fill=NA)
gg <- gg + coord_proj("+proj=robin +lon_0=0 +x_0=0 +y_0=0 +ellps=WGS84 +datum=WGS84 +units=m +no_defs")
gg <- gg + theme_map()

set.seed(1234)
df <- data.frame(value =round(c(rnorm(200000,
                                      mean=0,
                                      sd=1))))

norberg <- function(a,b,width,opt,Temps){
    all_nb=c()
    for (Temp in Temps) {
        if (a*exp(b*Temp)*(1-((Temp-opt)/(width/2))^2) > 0) {
            all_nb=c(all_nb,a*exp(b*Temp)*(1-((Temp-opt)/(width/2))^2))
        } else {
            all_nb=c(all_nb,0)
        }
    }
    return(paste(all_nb,collapse=","))
}

temp_vect=seq(-5,35,length.out=200)
merged_w_vals=tracer_fr_more_widths %>% rowwise() %>% dplyr::mutate(norberg_vals=norberg(a,b,w,opt,temp_vect),
                                                               temps=paste(temp_vect,collapse=",")) %>% 
    tidyr::separate_rows(norberg_vals,temps,sep=",")

merged_w_vals=merged_w_vals %>% rowwise() %>% dplyr::mutate(Tracer=factor(paste(w,opt,sep="_")))%>% 
    ungroup() %>%
    filter(Tracer %in% sample(levels(Tracer),4))

selection_plot=ggplot(merged_w_vals) + 
    geom_line(aes(x=as.numeric(temps),y=as.numeric(norberg_vals),group=Tracer)) + 
    theme_bw(base_size=12) + xlab("Temperature") + ylab("Growth rate")

# create density plot
density_plot=ggplot(df, aes(x=value)) + geom_density(adjust=15) + 
    geom_vline(aes(xintercept=0.5),color="red",linetype="dashed")+
    theme_void(base_size=12) + ggtitle("Distribution of maxima")

density_plot_2=ggplot(df, aes(x=value)) + geom_density(adjust=15) + 
    geom_vline(aes(xintercept=-2.5),color="red",linetype="dashed")+
    theme_void(base_size=12) + ggtitle("Distribution of thermal minima") 

gg|(selection_plot/(density_plot_2|density_plot)) #) + plot_layout(widths=c(1,0.3),heights=c(1,1))


In [ ]:
gg/selection_plot/(density_plot_2|density_plot) + plot_layout(heights=c(1,0.7,0.7))
ggsave("selection_criteria.pdf")